In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


In [ ]:
class ReStraVClassifier(nn.Module):
    def __init__(self, input_dim=21, h1=64, h2=32):
        """
        Defines the 2-layer MLP classifier as described in the paper.
        """
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, h1), 
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(h1, h2),         
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(h2, 1)        
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.network(x)

print("ReStraVClassifier class defined.")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

data_path = "D:\AI Video detector\data\processed\features.csv"
try:
    df = pd.read_csv(data_path)
    print(f"Successfully loaded data from {data_path}")
    print(f"Total samples: {len(df)}")
except FileNotFoundError:
    print(f"ERROR: File not found at {data_path}")
    print("Please make sure you ran the processing notebook first.")

feature_columns = [col for col in df.columns if col.startswith('f_')]
X = df[feature_columns].values
y = df['label'].values

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
print("Data scaled successfully.")

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1).to(device)

print("Data successfully converted to PyTorch tensors and sent to device.")